In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# RFM ANALİZİ

In [9]:
def rfm():
    #Tum kolonların gosterilmesini saglar
    pd.set_option('display.max_columns', None) 
    df = pd.read_excel("../input/turkish-market-sales-dataset-with-9000items/MarketSales.xlsx")
    df.head()

    #Tarihlerin tekil sayısını verir.
    df["STARTDATE"].nunique()

    #Eksik gozlemleri verir.
    df.isnull().sum()

    #veri boyutunun sorgusu
    df.shape

    # essiz urun sayisi
    df.CATEGORY_NAME1.nunique()
    df.CATEGORY_NAME2.nunique()
    df.CATEGORY_NAME3.nunique()

    #Essiz 354 adet urun bulunmaktadır.
    df.BRAND.nunique()

    # urunlerin kaç kere gectigini gösteriyor
    df.CATEGORY_NAME3.value_counts().head(30)

    # en cok siparis edilen urun hangisidir ? 
    # iptallerden dolayı eksi deger gozukuyor
    df.groupby("CATEGORY_NAME3") .agg({"AMOUNT":"sum"}).head()

    #En cok hangi ürün alınmıs.
    df.groupby("CATEGORY_NAME3").agg({"AMOUNT":"sum"}).sort_values("AMOUNT" ,ascending = False)

    #Toplamda odenen ucret
    df["TotalPrice"] = df["AMOUNT"] * df["PRICE"]
    df.head()

    #Faturaya gore en fazla odenen ucret.
    df.groupby("FICHENO").agg({"TotalPrice":"sum"}).head()

    # en fazla alışveriş yapan sehir
    df.CITY.value_counts()[0:5]

    # En fazla para harcayan sehir
    df.groupby("CITY").agg({"TotalPrice":"sum"}).sort_values("TotalPrice" , ascending = False).head()

    # Çeyreklıklerinin gosterilmesi
    df.describe([0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99]).T

    # Aykırı olan degerlerin gosterimi.
    for feature in ["AMOUNT","PRICE","TotalPrice"]:

        Q1 = df[feature].quantile(0.01)
        Q3 = df[feature].quantile(0.99)
        IQR = Q3-Q1
        upper = Q3 + 1.5*IQR
        lower = Q1 - 1.5*IQR

        if df[(df[feature] > upper) | (df[feature] < lower)].any(axis=None):
            print(feature,"yes")
            print(df[(df[feature] > upper) | (df[feature] < lower)].shape[0])
        else:
            print(feature, "no")


    # Recency
    #En son alısveris yapılan tarih.
    df["STARTDATE"].max()

    # Bugunun tarihi olarak en son gecen tarihi ekledim.
    import datetime as dt
    today_date = dt.datetime(2017 , 11 ,16)
    today_date

    # Musterilerin en son ne zaman alısveris yaptıga bakıyoruz.
    df.groupby("CLIENTCODE").agg({"STARTDATE":"max"}).head()

    #Musterilerin en son alısveris yaptıgı tarihten bugunu cıkarıyoruz.
    temp_df = (today_date - df.groupby("CLIENTCODE").agg({"STARTDATE":"max"}))

    #Degisken ismini degistirdim.
    temp_df.rename(columns = {"STARTDATE":"Recency"} , inplace= True)

    #Sadece gün sayısını elde ettik.
    recency_df = temp_df.Recency.apply(lambda x : x.days)

    # Frequency (sıklık)
    freq_df = df.groupby("CLIENTCODE").agg({"FICHENO":"nunique"})

    #Degisken ismini degistirdim.
    freq_df.rename(columns = {"FICHENO" : "Frequency"} , inplace = True)

    # Monetary (musterinin toplam bıraktıgı para mıktarı)
    monetary_df = df.groupby("CLIENTCODE").agg({"TotalPrice":"sum"})
    monetary_df.head()

    #Degisken ismini degistirdim.
    monetary_df.rename(columns= {"TotalPrice":"Monetary"} , inplace = True)

    #Degiskenlerin birlestirilmesi
    rfm = pd.concat([recency_df , freq_df , monetary_df] , axis = 1)

    # Skorlama işlemi
    rfm["RecencyScore"] = pd.qcut(rfm["Recency"], 5, labels = [5, 4 , 3, 2, 1])
    rfm.head()
    rfm["Recency"].min()
    rfm["Frequency"].min()

    # Skorlama işlemi
    rfm["FrequencyScore"]= pd.qcut(rfm["Frequency"].rank(method="first"),5, labels=[1,2,3,4,5])

    # Skorlama işlemi
    rfm["MonetaryScore"] = pd.qcut(rfm["Monetary"] , 5 , labels = [1,2,3,4,5])

    rfm = rfm.dropna()

    # rfm skorları kategorik degere donusturuldu
    rfm["rfm_score"] = (rfm.RecencyScore.astype(str)+
                        rfm.FrequencyScore.astype(str) +
                        rfm.MonetaryScore.astype(str))

    #En cok sipariş veren ve sitede aktif olan kişilerin getirlmesi.
    rfm.loc[rfm.rfm_score=="555"]

    # Regular Expressions (Düzenli İfadeler) kullanılarak RFM haritası çıkarıldı
    seg_map = {
        r'[1-2][1-2]': 'Hibernating',
        r'[1-2][3-4]': 'At Risk',
        r'[1-2]5': 'Can\'t Loose',
        r'3[1-2]': 'About to Sleep',
        r'33': 'Need Attention',
        r'[3-4][4-5]': 'Loyal Customers',
        r'41': 'Promising',
        r'51': 'New Customers',
        r'[4-5][2-3]': 'Potential Loyalists',
        r'5[4-5]': 'Champions'
    }

    #Monetary degeri dısarıda bırakılarak ,  FrequencyScore ve RecencyScore degerlerinin birlestirilerek segment isminde degisken olusturulması.
    rfm["segment"] =rfm.RecencyScore.astype(str) + rfm.FrequencyScore.astype(str)
    rfm.head() 

    #OLusturdugumuz regex sozlugunun tum segment'e uygulanarak isimlendirdim.
    rfm.segment = rfm.segment.replace(seg_map , regex = True)

    #df ve rfm'in birlestilmesi
    result = pd.merge(df, rfm, on='CLIENTCODE')
    
    #Segment'e gore grupların toplam harcamasının medyan degeri.
    #result.groupby("segment").agg({"TotalPrice":np.median})

    # Segmentlerin recency , frequency ve monetary degerlerine gore ortalama ve medyanın incelenmesi.
    #rfm[["segment","Recency","Frequency","Monetary"]].groupby("segment").agg(["mean","median","count"])
    return result

In [10]:
rfm()

AMOUNT yes
996
PRICE yes
75
TotalPrice yes
613


,ID,ITEMCODE,ITEMNAME,FICHENO,DATE_,AMOUNT,PRICE,LINENETTOTAL,LINENET,BRANCHNR,BRANCH,SALESMAN,CITY,REGION,LATITUDE,LONGITUDE,CLIENTCODE,CLIENTNAME,BRANDCODE,BRAND,CATEGORY_NAME1,CATEGORY_NAME2,CATEGORY_NAME3,STARTDATE,ENDDATE,GENDER,TotalPrice,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,rfm_score,segment
0,11738.0,5863.0,SPRITE 1 LT LIMON AROMALI GAZOZ,18456,2017-01-07,1.00,2.00,2.00,1.85,52.0,Kocaeli Subesi,Eyüp CİNE,Kocaeli,Marmara,40.8533,29.8815,467369,Sercan KIZILOK,156,SPRİTE,İÇECEK,GAZLI İÇECEK,GAZOZ,2017-01-08 16:16:11,2017-01-08 16:17:13,E,2.0000,244,3,93.2403,3,3,5,335,Need Attention
1,11741.0,11022.0,SEK GUNLUK SUT KUTU 500 ML.,18456,2017-01-07,1.00,1.79,1.79,1.66,52.0,Kocaeli Subesi,Eyüp CİNE,Kocaeli,Marmara,40.8533,29.8815,467369,Sercan KIZILOK,130,SEK,SÜT KAHVALTILIK,SÜT,GÜNLÜK SÜT,2017-01-08 16:16:11,2017-01-08 16:17:13,E,1.7900,244,3,93.2403,3,3,5,335,Need Attention
2,11750.0,3334.0,F SAFF ÇÖP POŞETİ JUMBO BOY,18456,2017-01-07,1.00,2.50,2.50,2.12,52.0,Kocaeli Subesi,Eyüp CİNE,Kocaeli,Marmara,40.8533,29.8815,467369,Sercan KIZILOK,50,F SAFF,EV,MUTFAK EŞYA GEREÇLERİ,MUTFAK SARF MALZEMESİ,2017-01-08 16:16:11,2017-01-08 16:17:13,E,2.5000,244,3,93.2403,3,3,5,335,Need Attention
3,11734.0,21683.0,CEVIZ ICI OZEL,18456,2017-01-07,0.18,38.50,6.93,6.41,52.0,Kocaeli Subesi,Eyüp GÖKMENOĞLU,Kocaeli,Marmara,40.8533,29.8815,467369,Sercan KIZILOK,8,ARMONİ,GIDA,BÜSKİVİ ÇEREZ,KURUYEMİŞ,2017-01-08 16:16:11,2017-01-08 16:17:13,E,6.9300,244,3,93.2403,3,3,5,335,Need Attention
4,11745.0,3581.0,ETI HOSBES MANDALINA 80 GR.,18456,2017-01-07,1.00,1.00,1.00,0.93,52.0,Kocaeli Subesi,Eyüp GÖKMENOĞLU,Kocaeli,Marmara,40.8533,29.8815,467369,Sercan KIZILOK,44,ETİ,GIDA,BÜSKİVİ ÇEREZ,BÜSKİVİ,2017-01-08 16:16:11,2017-01-08 16:17:13,E,1.0000,244,3,93.2403,3,3,5,335,Need Attention
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592755,596528.0,13486.0,ANKARA MAKARNA BURGU 500GR,154571,2017-03-30,1.00,1.10,1.10,1.02,43.0,Karabük Subesi,Raziye MEŞECİ,Karabük,Karadeniz,41.2061,32.6204,549252,Rahime DENİZCİ,255,ANKARA,GIDA,MAKARNA,MAKARNA,2017-03-31 13:42:06,2017-03-31 13:42:26,K,1.1000,229,1,2.2000,5,1,1,511,New Customers
592756,604598.0,5701.0,PORTAKAL,156743,2017-03-31,2.15,1.95,4.19,3.88,40.0,İstanbul Subesi,Yunus Emre AVUZ,İstanbul,Marmara,41.0053,28.9770,483151,Batuhan KAÇAROĞLU,A25,HAL,MEYVE SEBZE,MEYVE,NaN,2017-04-01 16:45:10,2017-04-01 16:45:15,E,4.1925,228,1,4.1925,5,1,1,511,New Customers
592757,558736.0,873.0,HALK SUT 1/1 LT,145695,2017-03-24,1.00,2.10,2.10,1.94,63.0,Ordu Subesi,Dilek DÖŞEME,Ordu,Karadeniz,40.9839,37.8764,421690,Elif ada HASYÖRE,59,HALK,SÜT KAHVALTILIK,SÜT,PASTÖRİZE SÜT,2017-03-25 19:24:51,2017-03-25 19:24:57,K,2.1000,235,1,2.1000,4,1,1,411,Promising
592758,604571.0,5518.0,MAYDANOZ,156737,2017-03-31,1.00,0.50,0.50,0.46,71.0,Şanlıurfa Subesi,Elifsu ŞAHİNKAYA,Şanlıurfa,Güneydoğu Anadolu,37.1591,38.7969,134086,Engin GÜLEÇ,A25,HAL,MEYVE SEBZE,SEBZE,NaN,2017-04-01 16:18:14,2017-04-01 16:18:15,E,0.5000,228,1,0.5000,5,1,1,511,New Customers


# Birliktelik Analizi

In [18]:
# BİR SUBENİN TUM SEPETLERİNE YAPILAN BİRLİKTELİK ANALİZİDİR.

def normal( support , threshold):
    """
    support = Birliktelik analizinde kullanılan "min_support" degeri
    threshold = Birliktelik analizinde kullanılan "association_rules" fonksiyonun degeri.
    """
    #Kutuphanelerin yuklenmesi
    import matplotlib.pyplot as plt
    import pandas as pd
    import seaborn as sns
   
    #Verinin fonksiyona tanımlanması
    def satislar_load():
        dff =pd.read_excel("../input/turkish-market-sales-dataset-with-9000items/MarketSales.xlsx")
        return dff
    
    #Veri setinin atanması
    df = satislar_load()
    
    #Veride bulunan sehirlerden birinin secilmesi
    df = df.loc[df.BRANCH == "İstanbul Subesi"]


    #Verideki tarih kısmını gün , ay , yıl olarak ayırılması
    df["STARTDATE"] = df["STARTDATE"].astype('datetime64[ns]')
    df["year"] =df["STARTDATE"].dt.year
    df["month"] =df["STARTDATE"].dt.month
    df["day"] =df["STARTDATE"].dt.day
    df["day_name"]= df.STARTDATE.dt.day_name()

    #df'in yedeginin alınması
    df_yedek = df.copy()

    #Urunlerin tekil isimlerinin alınması
    df_urunler = df_yedek.CATEGORY_NAME3.unique()

    #tum zamanlara gore urunlerin satısı
    #for a in df_urunler:
        #for i in ["day"  ,"day_name"]:
            #sns.countplot(df_yedek.loc[df_yedek.CATEGORY_NAME3 == a, "CATEGORY_NAME3"], hue=df_yedek[i])
            #plt.show()


    ### birliktelik analizi
    df_genel = df_yedek.copy()
     
    #Urunlerın virgul ile ayrılması
    df_yedek = df_yedek.CATEGORY_NAME3.str.strip(",")
    
    #Verilerde birlestirme isleminin yapılması
    dff = pd.concat([df_genel.FICHENO , df_yedek] , axis = 1 )
    
    #Eksik degerlerin dusurulmesi
    dff = dff.dropna()

    #Her kesilen faturada birlikte alınan urunlerin tek satırda virgulle birlestirilmesi
    dff = dff.groupby('FICHENO')['CATEGORY_NAME3'].agg(','.join).reset_index()
    
    #FICHENO degiskeninin dusurulmesi
    dff = dff.drop("FICHENO" , axis = 1)
    
    #Virgule gore urunlerin ayrılması ıslemi
    data = list(dff['CATEGORY_NAME3'].apply(lambda x:x.split(",")))
    

    #Enkod isleminin yapılması
    from mlxtend.preprocessing import TransactionEncoder
    tencoder = TransactionEncoder()
    te_data = tencoder.fit(data).transform(data)
    df = pd.DataFrame(te_data, columns=tencoder.columns_)
   

    #Apriori algoritmasının kullanılması
    from mlxtend.frequent_patterns import apriori,  association_rules
    df1 = apriori(df, min_support=support, use_colnames=True)
   
    #Birliktelik kurallarının cıkarılması
    df_association = association_rules(df1, metric = 'confidence', min_threshold =threshold)
    
    return df_association.sort_values(by='confidence', ascending=False).reset_index()



In [19]:
normal( 0.01 , 0.5)

,index,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,0,(KEK),(BÜSKİVİ),0.060163,0.192295,0.033021,0.548866,2.854293,0.021452,1.790389


**GUN OLARAK DEGERLENDİRME**

In [4]:
# GUN OLARAK DEGERLENDİRME
def day( support , threshold , day , plot = False ):
    """
    support = Birliktelik analizinde kullanılan "min_support" degeri
    threshold = Birliktelik analizinde kullanılan "association_rules" fonksiyonun degeri.
    day = Secilecek gun bazında birliktelik analizi uygular.
    plot = "True" yapıldıgında urunlerın gun bazlı grafiklerini verir.
    """
    #Kutuphanelerin yuklenmesi
    import matplotlib.pyplot as plt
    import pandas as pd
    import seaborn as sns
    
    #Verinin fonksiyona tanımlanması
    def satislar_load():
        dff =pd.read_excel("../input/turkish-market-sales-dataset-with-9000items/MarketSales.xlsx")
        return dff
    
    #Veri setinin atanması
    df = satislar_load()
    
    #Veride bulunan sehirlerden birinin secilmesi
    df = df.loc[df.BRANCH == "İstanbul Subesi"]


    ## ucuncu. adım

    #Verideki tarih kısmını gün , ay , yıl olarak ayırılması
    df["STARTDATE"] = df["STARTDATE"].astype('datetime64[ns]')
    df["year"] =df["STARTDATE"].dt.year
    df["month"] =df["STARTDATE"].dt.month
    df["day"] =df["STARTDATE"].dt.day
    df["day_name"]= df.STARTDATE.dt.day_name()


    #df'in yedeginin alınması
    df_yedek = df.copy()
    
    #Secilecek gun uzerinden ıslemler yapılacaktır
    df_yedek = df_yedek.loc[df_yedek.day_name == day]
    
    #Urunlerin tekil isimlerinin alınması
    df_urunler = df_yedek.CATEGORY_NAME3.unique()

    #tum zamanlara gore urunlerin satısı
    
    if plot:
        for a in df_urunler:
            for i in ["hour" ,"day_name"]:
                sns.countplot(df_yedek.loc[df_yedek.CATEGORY_NAME3 == a, "CATEGORY_NAME3"], hue=df_yedek[i])
                plt.show()

    ### birliktelik analizi
    df_genel = df_yedek.copy()
    
    #Urunlerın virgul ile ayrılması
    df_yedek = df_yedek.CATEGORY_NAME3.str.strip(",")

    #Verilerde birlestirme isleminin yapılması
    dff = pd.concat([df_genel.FICHENO , df_yedek] , axis = 1 )
    
    #Eksik degerlerin dusurulmesi
    dff = dff.dropna()

    #Her kesilen faturada birlikte alınan urunlerin tek satırda virgulle birlestirilmesi
    dff = dff.groupby('FICHENO')['CATEGORY_NAME3'].agg(','.join).reset_index()

    #FICHENO degiskeninin dusurulmesi
    dff = dff.drop("FICHENO" , axis = 1)
    
    #Virgule gore urunlerin ayrılması ıslemi
    data = list(dff['CATEGORY_NAME3'].apply(lambda x:x.split(",")))
    

    #Enkod isleminin yapılması
    from mlxtend.preprocessing import TransactionEncoder
    tencoder = TransactionEncoder()
    te_data = tencoder.fit(data).transform(data)
    df = pd.DataFrame(te_data, columns=tencoder.columns_)
   

    #Apriori algoritmasının kullanılması
    from mlxtend.frequent_patterns import apriori,  association_rules
    df1 = apriori(df, min_support=support, use_colnames=True)
   
    #Birliktelik kurallarının cıkarılması
    df_association = association_rules(df1, metric = 'confidence', min_threshold =threshold)
    
    return df_association.sort_values(by='confidence', ascending=False).reset_index()

In [11]:
day( 0.01 , 0.50 , "Monday" )

,index,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,1,"(MAKARNA , SEKER TATLANDIRICI)",(AÇIK BAKLİYAT),0.015947,0.094352,0.010631,0.666667,7.065728,0.009127,2.716944
1,3,"(SEKER TATLANDIRICI, AÇIK BAKLİYAT)",(MAKARNA ),0.017276,0.066113,0.010631,0.615385,9.308079,0.009489,2.428106
2,0,(KEK),(BÜSKİVİ),0.060133,0.192359,0.034551,0.574586,2.987051,0.022984,1.898481
3,2,"(MAKARNA , AÇIK BAKLİYAT)",(SEKER TATLANDIRICI),0.020598,0.078073,0.010631,0.516129,6.610844,0.009023,1.905316


In [12]:
day( 0.01 , 0.50 , "Tuesday" )

,index,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,0,(KEK),(BÜSKİVİ),0.058246,0.18896,0.034541,0.593023,3.138347,0.023535,1.99284


In [13]:
day( 0.01 , 0.50 , "Wednesday" )

,index,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,3,"(ÇAY, AÇIK BAKLİYAT)",(SEKER TATLANDIRICI),0.018502,0.081645,0.011454,0.619048,7.582220,0.009943,2.410683
1,1,"(MAKARNA , MARGARİNLER)",(AÇIK BAKLİYAT),0.017327,0.110426,0.010279,0.593220,5.372115,0.008366,2.186870
2,4,"(MARGARİNLER, SEKER TATLANDIRICI)",(BÜSKİVİ),0.017621,0.194420,0.010279,0.583333,3.000378,0.006853,1.933392
3,0,(KEK),(BÜSKİVİ),0.066079,0.194420,0.037885,0.573333,2.948943,0.025038,1.888078
4,2,"(MAKARNA , SEKER TATLANDIRICI)",(AÇIK BAKLİYAT),0.019090,0.110426,0.010573,0.553846,5.015548,0.008465,1.993873


In [14]:
day( 0.01 , 0.50 , "Thursday" )

,index,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,10,"(ÇAY, AÇIK BAKLİYAT)",(SEKER TATLANDIRICI),0.014268,0.075535,0.011330,0.794118,10.513235,0.010253,4.490258
1,16,"(ÇAY, MAKARNA )",(SEKER TATLANDIRICI),0.018044,0.075535,0.013428,0.744186,9.852196,0.012065,3.613818
2,9,"(ÇAY, AÇIK BAKLİYAT)",(MAKARNA ),0.014268,0.073856,0.010491,0.735294,9.955715,0.009437,3.498764
3,4,"(AÇIK BAKLİYAT, PASTÖRİZE SÜT)",(MAKARNA ),0.015107,0.073856,0.010491,0.694444,9.402620,0.009375,3.031015
4,11,"(KEK, PASTÖRİZE SÜT)",(BÜSKİVİ),0.015527,0.188418,0.010071,0.648649,3.442605,0.007146,2.309887
5,12,"(MAKARNA , PASTÖRİZE SÜT)",(BÜSKİVİ),0.021821,0.188418,0.013428,0.615385,3.266061,0.009317,2.110113
6,15,"(SEKER TATLANDIRICI, PASTÖRİZE SÜT)",(MAKARNA ),0.017205,0.073856,0.010491,0.609756,8.255959,0.009220,2.373243
7,18,"(MAKARNA , SEKER TATLANDIRICI)",(ÇAY),0.022241,0.062946,0.013428,0.603774,9.591950,0.012028,2.364946
8,5,"(MAKARNA , SEKER TATLANDIRICI)",(AÇIK BAKLİYAT),0.022241,0.084767,0.013428,0.603774,7.122735,0.011543,2.309874
9,8,"(ÇAY, MAKARNA )",(AÇIK BAKLİYAT),0.018044,0.084767,0.010491,0.581395,6.858738,0.008961,2.186390


In [15]:
day( 0.01 , 0.50 , "Friday" )

,index,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,0,(KEK),(BÜSKİVİ),0.062173,0.200369,0.033857,0.544554,2.717753,0.021399,1.755711


In [16]:
day( 0.01 , 0.50 , "Saturday" )

,index,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,1,(KREM ÇİKOLATA),(BÜSKİVİ),0.034744,0.198358,0.018004,0.518182,2.612362,0.011112,1.663786
1,0,(KEK),(BÜSKİVİ),0.059381,0.198358,0.030322,0.510638,2.574333,0.018544,1.638139


In [17]:
day( 0.01 , 0.50 , "Sunday" )

,index,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,1,"(MAKARNA , SEKER TATLANDIRICI)",(AÇIK BAKLİYAT),0.014226,0.101196,0.010023,0.704545,6.962170,0.008583,3.042105
1,0,(KEK),(BÜSKİVİ),0.052376,0.181377,0.026511,0.506173,2.790718,0.017012,1.657711
2,2,"(SEKER TATLANDIRICI, AÇIK BAKLİYAT)",(MAKARNA ),0.020045,0.070158,0.010023,0.500000,7.126728,0.008616,1.859683


# CLTV

In [ ]:
#CLTV
#import modules
import pandas as pd # for dataframes
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
import datetime as dt
import numpy as np

In [5]:
def cltv():
    
    def satislar_load():
        dff =pd.read_excel("../input/turkish-market-sales-dataset-with-9000items/MarketSales.xlsx")
        return dff
    
    df = satislar_load()
    df.head()

    #Calulate total purchase
    df['TotalPurchase'] = df['AMOUNT'] * df['PRICE']

    df=df.groupby('CLIENTCODE').agg({'STARTDATE': lambda date: (date.max() - date.min()).days,
                                            'FICHENO': lambda num: len(num),
                                            'AMOUNT': lambda quant: quant.sum(),
                                            'TotalPurchase': lambda price: price.sum()})

    # Change the name of columns
    df.columns=['num_days','num_transactions','num_units','spent_money']
    df.head()


    #1. Calculate Average Order Value
   
    # Average Order Value
    df['avg_order_value']=df['spent_money']/df['num_transactions']

    #2. Calculate Purchase Frequency

    purchase_frequency=sum(df['num_transactions'])/df.shape[0]

    #3. Calculate Repeat Rate and Churn Rate

    # Repeat Rate
    repeat_rate=df[df.num_transactions > 1].shape[0]/df.shape[0]

    #Churn Rate
    churn_rate=1-repeat_rate

    # Profit Margin(Kar marjı)
    df['profit_margin']=df['spent_money']*0.10

    # Customer Value
    df['CLV']=(df['avg_order_value']*purchase_frequency)/churn_rate

    #Customer Lifetime Value
    df['cust_lifetime_value']=df['CLV']*df['profit_margin']
    return df

In [8]:
cltv()

,num_days,num_transactions,num_units,spent_money,avg_order_value,profit_margin,CLV,cust_lifetime_value
CLIENTCODE,,,,,,,,
1,30,25,26.438,115.58950,4.623580,11.558950,818.355712,9459.332762
10,55,2,2.000,4.25000,2.125000,0.425000,376.116751,159.849619
100,3,8,25.759,297.81975,37.227469,29.781975,6589.117461,196236.931490
1000,14,6,7.320,31.28600,5.214333,3.128600,922.916759,2887.437373
1000035,0,2,2.000,16.00000,8.000000,1.600000,1415.968946,2265.550314
...,...,...,...,...,...,...,...,...
999955,47,6,5.070,19.35275,3.225458,1.935275,570.893605,1104.836121
999965,0,1,1.000,1.00000,1.000000,0.100000,176.996118,17.699612
999966,43,8,9.377,48.58905,6.073631,4.858905,1075.009155,5223.367358
